0: fake
1: real
|561.605

In [30]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.feature_extraction.text import TfidfVectorizer
from transformers import PhobertTokenizer, TFRobertaModel
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Bidirectional
from tensorflow.keras.optimizers import Adam

In [82]:
tf.config.set_visible_devices([], 'GPU')

RuntimeError: Visible devices cannot be modified after being initialized

In [5]:
# Tải tokenizer và mô hình PhoBERT
tokenizer = PhobertTokenizer.from_pretrained("vinai/phobert-base")
phobert_model = TFRobertaModel.from_pretrained("vinai/phobert-base")

Some layers from the model checkpoint at vinai/phobert-base were not used when initializing TFRobertaModel: ['lm_head']
- This IS expected if you are initializing TFRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFRobertaModel were initialized from the model checkpoint at vinai/phobert-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaModel for predictions without further training.


In [31]:
import numpy as np

# Đường dẫn đến file .npy
duong_dan_file = '/Users/quocminh/Desktop/Đồ Án/chinh thuc/data/data vlsp/data_goc/process/embeddings_array_train.npy'

# Sử dụng np.load() để đọc ma trận từ file
# ma_tran_du_lieu = np.load(duong_dan_file)
phobert_embeddings = np.load(duong_dan_file)
# Hiển thị ma trận
print("Ma trận đọc từ file:")
print(phobert_embeddings)


Ma trận đọc từ file:
[[[-0.6359838   1.1019332  -0.7713704  ... -0.1958782  -0.57587266
    0.0297346 ]
  [-0.06781114  0.7031037  -0.518356   ...  0.41271758  0.03433367
   -0.55555195]
  [-0.13347109  0.4953625  -0.7996192  ... -0.26969117 -0.10532248
   -0.23023811]
  ...
  [ 0.10418507  0.3340481  -0.6007942  ... -0.3739854  -0.02259465
    0.01876782]
  [-0.4236371   0.50411725 -0.25576037 ... -0.43336388 -0.36910945
    0.20797461]
  [-0.7399383   1.0313084  -0.6763179  ... -0.01023581 -0.67881626
    0.11209171]]

 [[-0.33545628  0.62904996 -0.60383403 ...  0.48500472 -0.4199828
   -0.30491263]
  [-0.1880203   0.04489954 -0.04217417 ...  0.28976932 -0.44473907
   -0.25600773]
  [-0.40234983  0.47170758  0.20504543 ...  0.34907067 -0.24179958
    0.1116198 ]
  ...
  [ 0.          0.          0.         ...  0.          0.
    0.        ]
  [ 0.          0.          0.         ...  0.          0.
    0.        ]
  [ 0.          0.          0.         ...  0.          0.
    0.    

In [32]:
phobert_embeddings.shape

(6359, 106, 768)

In [47]:
# Đọc dữ liệu
df = pd.read_csv("/Users/quocminh/Desktop/Đồ Án/chinh thuc/data/data vlsp/data_goc/process/train_new.csv")

In [48]:
df.head()

,Unnamed: 0,user_name,post_message,timestamp_post,num_like_post,num_comment_post,num_share_post,label,token,token_post_message
0,0,f3317e52aa79b261db534a38e7f7c360,ĐẾN 18H CHIỀU 10/4 : THÊM 2 CA MẮC COVID-19 ➡️...,1586491781,5505,34,374.0,0,18h chiều 104 2 ca mắc covid19 bn 256 nam 52 q...,18h chiều 104 2 ca mắc covid19 bn 256 nam 52 q...
1,1,cb6ad3159dd0f8cc5fa2833b22ed3c90,Thế giới tuần qua: Nợ công Mỹ tăng kinh hoàng ...,2020-06-13 02:21:21,8.0,2.0,4.0,0,thế_giới tuần nợ công mỹ kinh_hoàng 26000 tỷ u...,thế_giới tuần nợ công mỹ kinh_hoàng 26000 tỷ u...
2,2,0f93c7b7aedee5cd1c69cfa61787d586,Sự_cố đứt cáp AAG không ảnh_hưởng tới chất_lượ...,1585954578,2,0,2.0,0,sự_cố đứt cáp aag ảnh_hưởng chất_lượng trang i...,sự_cố đứt cáp aag ảnh_hưởng chất_lượng trang i...
3,3,b008022f7508beb6360b691c8c9461e5,TRIỆU CHỨNG NHIỄM CORONA QUA TỪNG NGÀY 🛑Ngày 1...,2020-03-01 18:18:51,1000.0,155.0,3000.0,1,triệu_chứng nhiễm corona ngày_ngày 1 3 triệu_c...,triệu_chứng nhiễm corona ngày_ngày 1 3 triệu_c...
4,4,65c95e0460af8d14665f73aee98aec88,TỔN THƯƠNG PHỔI CỦA BỆNH NHÂN ĐÃ CHIẾM 90%\n\n...,2020-05-13 02:45:40,5344.0,50.0,88.0,0,tổn_thương phổi bệnh_nhân chiếm 90 hội_chẩn li...,tổn_thương phổi bệnh_nhân chiếm 90 hội_chẩn li...


In [73]:
data = phobert_embeddings
label = df['label']

In [75]:
# Flatten mảng 3D thành mảng 2D
data_2d = data.reshape(data.shape[0], -1)

In [80]:
data_2d.shape

(6359, 81408)

In [59]:
data.shape

(6359,)

In [60]:
label.shape

(6359,)

In [53]:
data[2]

'sự_cố đứt cáp aag ảnh_hưởng chất_lượng trang internet ứng_dụng học_tập đào_tạo sử_dụng nền_tảng phần_mềm doanh_nghiệp việt_nam phát_triển server việt_nam'

In [63]:
from sklearn.feature_extraction.text import TfidfVectorizer
# Tạo vector hóa TF-IDF
vectorizer = TfidfVectorizer()
X_train_tfidf = vectorizer.fit_transform(data)

In [64]:
X_train_tfidf.shape

(6359, 23745)

In [78]:
from imblearn.over_sampling import SMOTE
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(data_2d, label)

In [79]:
X_train_resampled.shape

(10522, 81408)

In [71]:
y_train_resampled.shape

(10522,)

In [88]:
# Xác định kích thước của mảng 3D mới
num_samples = X_train_resampled.shape[0]
max_seq_length = 106  # Độ dài chuỗi tối đa cho mỗi mẫu trong mảng 3D mới
embedding_size = 768  # Kích thước của embedding cho mỗi phần tử trong chuỗi

# Sử dụng reshape để chuyển từ 2D sang 3D
data_3d = X_train_resampled.reshape(num_samples, max_seq_length, embedding_size)

In [90]:
data_3d.shape

(10522, 106, 768)

In [9]:
data[16]

array([[-0.15379545,  0.616008  , -0.6380138 , ..., -0.01640839,
        -0.27363673, -0.72130424],
       [-0.0289081 , -0.3191048 , -0.10481234, ...,  0.04813703,
        -0.09355595,  0.46457505],
       [ 0.31515437,  0.13389817,  0.11998722, ..., -0.05407841,
        -0.4661887 , -0.26282856],
       ...,
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ]], dtype=float32)

In [7]:
from sklearn.model_selection import train_test_split
X_train, X_rem, y_train, y_rem = train_test_split(data, label, train_size= 0.8)

# Now since we want the valid and test size to be equal (10% each of overall data).
# we have to define valid_size=0.5 (that is 50% of remaining data)
test_size = 0.5
X_valid, X_test, y_valid, y_test = train_test_split(X_rem,y_rem, test_size=0.5)

print(X_train.shape), print(y_train.shape)
print(X_valid.shape), print(y_valid.shape)
print(X_test.shape), print(y_test.shape)

(8000, 256, 768)
(8000,)
(1000, 256, 768)
(1000,)
(1000, 256, 768)
(1000,)


(None, None)

In [9]:
tf.config.set_visible_devices([], 'GPU')

In [91]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Bidirectional, LSTM, Dense, Dropout

# Kích thước của vector embedding PhoBERT
embedding_shape = (106, 768)

# Số lớp đầu ra (tùy thuộc vào số lượng nhãn trong dữ liệu của bạn)
num_classes = 1

# Tạo mô hình Bi-LSTM
model = Sequential()
model.add(Bidirectional(LSTM(128, return_sequences=True), input_shape=embedding_shape))
# model.add(Dropout(0.5))
model.add(Bidirectional(LSTM(64)))
# model.add(Dropout(0.5))
model.add(Dense(32, activation='relu'))
# model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='sigmoid'))

# Compile mô hình
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# In thông tin tóm tắt của mô hình
model.summary()


Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional_2 (Bidirecti  (None, 106, 256)          918528    
 onal)                                                           
                                                                 
 bidirectional_3 (Bidirecti  (None, 128)               164352    
 onal)                                                           
                                                                 
 dense (Dense)               (None, 32)                4128      
                                                                 
 dense_1 (Dense)             (None, 1)                 33        
                                                                 
Total params: 1087041 (4.15 MB)
Trainable params: 1087041 (4.15 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [92]:
# Huấn luyện mô hình trên dữ liệu của bạn (data và label)
from tensorflow.keras.callbacks import EarlyStopping
# Thay thế data và label bằng dữ liệu và nhãn thực tế của bạn
# Tự dừng học khi không cải thiện validation loss trong 10 epoch
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Huấn luyện mô hình với tự dừng học
# history = model.fit(X_train, y_train, epochs=100, validation_data=(x_val, y_val), callbacks=[early_stopping])

model.fit(data_3d, y_train_resampled, epochs=200, batch_size=32, validation_split=0.2, callbacks=[early_stopping])

Epoch 1/200


2024-02-03 13:04:03.261214: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


264/264 [==============================] - 30s 91ms/step - loss: 0.3329 - accuracy: 0.8543 - val_loss: 0.1645 - val_accuracy: 0.9520
Epoch 2/200
264/264 [==============================] - 21s 81ms/step - loss: 0.1541 - accuracy: 0.9410 - val_loss: 0.0246 - val_accuracy: 0.9933
Epoch 3/200
264/264 [==============================] - 20s 77ms/step - loss: 0.1325 - accuracy: 0.9501 - val_loss: 0.0337 - val_accuracy: 0.9933
Epoch 4/200
264/264 [==============================] - 20s 77ms/step - loss: 0.0564 - accuracy: 0.9818 - val_loss: 0.0704 - val_accuracy: 0.9729
Epoch 5/200
264/264 [==============================] - 20s 77ms/step - loss: 0.0372 - accuracy: 0.9885 - val_loss: 0.0072 - val_accuracy: 0.9967
Epoch 6/200
264/264 [==============================] - 20s 77ms/step - loss: 0.0245 - accuracy: 0.9922 - val_loss: 0.0393 - val_accuracy: 0.9862
Epoch 7/200
264/264 [==============================] - 20s 77ms/step - loss: 0.0152 - accuracy: 0.9956 - val_loss: 0.0031 - val_accuracy: 1.00

In [127]:
df_test = pd.read_csv('/Users/quocminh/Desktop/Đồ Án/chinh thuc/data/data vlsp/data_goc/process/test_new.csv')

In [128]:
import numpy as np

# Đường dẫn đến file .npy
duong_dan_file = '/Users/quocminh/Desktop/Đồ Án/chinh thuc/data/data vlsp/data_goc/process/embeddings_array_test.npy'

# Sử dụng np.load() để đọc ma trận từ file
# ma_tran_du_lieu = np.load(duong_dan_file)
phobert_embeddings_test = np.load(duong_dan_file)
# Hiển thị ma trận
print("Ma trận đọc từ file:")
print(phobert_embeddings)


Ma trận đọc từ file:
[[[-0.6359838   1.1019332  -0.7713704  ... -0.1958782  -0.57587266
    0.0297346 ]
  [-0.06781114  0.7031037  -0.518356   ...  0.41271758  0.03433367
   -0.55555195]
  [-0.13347109  0.4953625  -0.7996192  ... -0.26969117 -0.10532248
   -0.23023811]
  ...
  [ 0.10418507  0.3340481  -0.6007942  ... -0.3739854  -0.02259465
    0.01876782]
  [-0.4236371   0.50411725 -0.25576037 ... -0.43336388 -0.36910945
    0.20797461]
  [-0.7399383   1.0313084  -0.6763179  ... -0.01023581 -0.67881626
    0.11209171]]

 [[-0.33545628  0.62904996 -0.60383403 ...  0.48500472 -0.4199828
   -0.30491263]
  [-0.1880203   0.04489954 -0.04217417 ...  0.28976932 -0.44473907
   -0.25600773]
  [-0.40234983  0.47170758  0.20504543 ...  0.34907067 -0.24179958
    0.1116198 ]
  ...
  [ 0.          0.          0.         ...  0.          0.
    0.        ]
  [ 0.          0.          0.         ...  0.          0.
    0.        ]
  [ 0.          0.          0.         ...  0.          0.
    0.    

In [129]:
X_test = phobert_embeddings_test
y_test = df_test['label']

In [96]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Giả sử model là mô hình bạn đã huấn luyện và test_data là tập dữ liệu kiểm thử
test_predictions = model.predict(X_test)

# Chuyển đổi dự đoán thành nhãn (ví dụ: với mô hình binary classification)
predicted_labels = (test_predictions > 0.5).astype(int)

# Đánh giá mô hình
accuracy = accuracy_score(y_test, predicted_labels)
conf_matrix = confusion_matrix(y_test, predicted_labels)
classification_rep = classification_report(y_test, predicted_labels)

# In các độ đo
print("Accuracy:", accuracy)
print("Confusion Matrix:\n", conf_matrix)
print("Classification Report:\n", classification_rep)


11/11 [==============================] - 2s 77ms/step
Accuracy: 0.8424068767908309
Confusion Matrix:
 [[230  53]
 [  2  64]]
Classification Report:
               precision    recall  f1-score   support

           0       0.99      0.81      0.89       283
           1       0.55      0.97      0.70        66

    accuracy                           0.84       349
   macro avg       0.77      0.89      0.80       349
weighted avg       0.91      0.84      0.86       349



In [97]:
# Đánh giá mô hình trên tập kiểm tra
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Loss: {loss}, Accuracy: {accuracy}")

11/11 [==============================] - 0s 40ms/step - loss: 0.6360 - accuracy: 0.8424
Loss: 0.6360059380531311, Accuracy: 0.8424068689346313


In [102]:
# Đánh giá mô hình trên tập kiểm tra
loss, accuracy = model.evaluate(X_val, y_val)
print(f"Loss: {loss}, Accuracy: {accuracy}")


12/12 [==============================] - 1s 37ms/step - loss: 0.4631 - accuracy: 0.8952
Loss: 0.46305710077285767, Accuracy: 0.8951841592788696


In [ ]:
# # Biểu diễn mẫu tin tức bằng PhoBERT và lấy ra vector embedding
# phobert_inputs = tokenizer(data, return_tensors="tf", padding=True, truncation=True, max_length=250, is_split_into_words=False)
# phobert_outputs = phobert_model(phobert_inputs)
# phobert_embeddings = phobert_outputs.last_hidden_state

In [ ]:
# # Xây dựng mô hình
# model = Sequential()
# # tinh chỉnh lại tham số cho phù hợp
# model.add(Bidirectional(LSTM(512, return_sequences=True), input_shape=(128, 768))) #tunh chinh lai inputshape
# model.add(Bidirectional(LSTM(256)))
# model.add(Dense(128, activation='relu'))
# model.add(Dense(1, activation='sigmoid'))

# # Biên dịch mô hình
# model.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=0.001), metrics=['accuracy'])

In [ ]:
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Bidirectional, LSTM, Dense, Dropout
# from tensorflow.keras.optimizers import Adam

# # Xây dựng mô hình
# model = Sequential()
# # Thay đổi số đơn vị và thêm Dropout
# model.add(Bidirectional(LSTM(256, return_sequences=True), input_shape=(128, 768)))  # Thay đổi số đơn vị
# model.add(Dropout(0.2))  # Thêm lớp Dropout để ngăn overfitting
# model.add(Bidirectional(LSTM(128)))  # Thay đổi số đơn vị
# model.add(Dropout(0.2))  # Thêm lớp Dropout
# model.add(Dense(128, activation='relu'))
# model.add(Dense(1, activation='sigmoid'))

# # Biên dịch mô hình với learning rate thấp hơn và optimizer RMSprop
# model.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=0.0001), metrics=['accuracy'])

# # Kiểm tra cấu trúc mô hình
# model.summary()

In [ ]:
# # Chia dữ liệu thành tập huấn luyện và tập kiểm tra
# split_ratio = 0.8
# split_index = int(len(texts) * split_ratio)

# X_train = np.array(phobert_embeddings[:split_index])
# X_test = np.array(phobert_embeddings[split_index:])
# y_train = np.array(labels[:split_index])
# y_test = np.array(labels[split_index:])

# # Huấn luyện mô hình
# model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)

Epoch 1/10
20/20 [==============================] - 61s 2s/step - loss: 0.4381 - accuracy: 0.8119 - val_loss: 0.4191 - val_accuracy: 0.8250
Epoch 2/10
20/20 [==============================] - 26s 1s/step - loss: 0.2443 - accuracy: 0.9028 - val_loss: 0.3787 - val_accuracy: 0.8562
Epoch 3/10
20/20 [==============================] - 23s 1s/step - loss: 0.1547 - accuracy: 0.9389 - val_loss: 0.4328 - val_accuracy: 0.8125
Epoch 4/10
20/20 [==============================] - 26s 1s/step - loss: 0.0637 - accuracy: 0.9812 - val_loss: 0.4558 - val_accuracy: 0.8750
Epoch 5/10
20/20 [==============================] - 23s 1s/step - loss: 0.0259 - accuracy: 0.9922 - val_loss: 0.6098 - val_accuracy: 0.8750
Epoch 6/10
20/20 [==============================] - 27s 1s/step - loss: 0.0468 - accuracy: 0.9859 - val_loss: 0.5386 - val_accuracy: 0.8250
Epoch 7/10
20/20 [==============================] - 23s 1s/step - loss: 0.0369 - accuracy: 0.9843 - val_loss: 0.5375 - val_accuracy: 0.8500
Epoch 8/10
20/20 [==

In [ ]:
# # Đánh giá mô hình trên tập kiểm tra
# loss, accuracy = model.evaluate(X_test, y_test)
# print(f"Loss: {loss}, Accuracy: {accuracy}")

4/4 [==============================] - 0s 49ms/step - loss: 0.0616 - accuracy: 0.9840
Loss: 0.06160259246826172, Accuracy: 0.984000027179718


In [27]:
from transformers import T5ForConditionalGeneration, T5Tokenizer, AutoModel, AutoTokenizer

In [28]:
def load_T5():
    t5_model = T5ForConditionalGeneration.from_pretrained("NlpHUST/t5-small-vi-summarization")
    t5_tokenizer = T5Tokenizer.from_pretrained("NlpHUST/t5-small-vi-summarization")
    return t5_model, t5_tokenizer
t5_model, t5_tokenizer = load_T5()

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [29]:
def summarize(sentence):
    with torch.no_grad():
        tokenized_text = t5_tokenizer.encode(sentence, return_tensors="pt")
        summary_ids = t5_model.generate(tokenized_text, max_length=256, num_beams=5,
                                     repetition_penalty=2.5, length_penalty=1.0, early_stopping=True)
    return t5_tokenizer.decode(summary_ids[0], skip_special_tokens=True)


In [106]:
def preprocess(text):
    
    text = re.sub(r'http\S+', '', text)

    text = re.sub(r'[^\w\s\d+]', '', text)

    text = text.lower()

    tokens = ViTokenizer.tokenize(text).split()

    stopwords_file = '/Users/quocminh/Desktop/Đồ Án/chinh thuc/final/stop_word.txt'
    with open(stopwords_file, 'r', encoding='utf-8') as f:
        stop_words = set([line.strip() for line in f])

    tokens = [token for token in tokens if token not in stop_words]

    tokens = [token.replace('\n', ' ') for token in tokens]

    text = ' '.join(tokens)

    return text

In [108]:
import torch
import re
from pyvi import ViTokenizer

In [115]:
def load_bert():
    tokenizer = PhobertTokenizer.from_pretrained("vinai/phobert-base")
    phobert_model = TFRobertaModel.from_pretrained("vinai/phobert-base")
    return tokenizer, phobert_model
tokenizer, phobert_model = load_bert()

Some layers from the model checkpoint at vinai/phobert-base were not used when initializing TFRobertaModel: ['lm_head']
- This IS expected if you are initializing TFRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFRobertaModel were initialized from the model checkpoint at vinai/phobert-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaModel for predictions without further training.


In [103]:
# Tokenize dữ liệu và lấy vector embedding
def phobert_process(data):
    phobert_inputs = tokenizer(data, return_tensors="tf", padding=True, truncation=True, max_length=106, is_split_into_words=False)
    phobert_outputs = phobert_model(phobert_inputs)
    phobert_embeddings = phobert_outputs.last_hidden_state
    return phobert_embeddings

In [169]:
sentence = '''
người trong hình tự thiêu do bức xúc cách chống dịch COVID-19
'''

In [170]:

result = phobert_process(preprocess((sentence)))
prediction = model.predict(result)
print(f"Predicted Label: {int(round(prediction[0][0]))}")
print(prediction[0][0])

if (round(prediction[0][0])) == 0:
    print('This is a real news.')
else:
    print('This is a fake news.')


1/1 [==============================] - 0s 47ms/step
Predicted Label: 0
0.0002354773
This is a real news.


In [131]:
from sklearn.metrics import f1_score

# Tính toán F1-score
test_predictions = model.predict(X_test)
# Chuyển đổi dự đoán thành nhãn (ví dụ: với mô hình binary classification)
predicted_labels = (test_predictions > 0.5).astype(int)
f1 = f1_score(y_test, predicted_labels)
print("F1-score:", f1)


11/11 [==============================] - 0s 28ms/step
F1-score: 0.6994535519125683


In [132]:
from sklearn.metrics import roc_auc_score

# Tính toán AUC
auc = roc_auc_score(y_test, predicted_labels)
print("AUC:", auc)


AUC: 0.8912089088767534


In [42]:
summarize(sentence)

'Những người đàn bà hoá đá sau một đêm đi vũ trường bay nhảy cùng các dân chơi trong xóm trọ. Nhiều người đàn bà hoá đá sau một đêm đi vũ trường bay nhảy cùng các dân chơi trong xóm trọ. Mấy ngày qua, tin nóng bàn hoàng dư luận mấy ngày qua.'

In [ ]:
print(prediction)

[[0.00694228]]


In [110]:
pip install joblib

python(9930) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


Note: you may need to restart the kernel to use updated packages.


In [111]:
import joblib

# Đường dẫn để lưu mô hình
model_path = "/Users/quocminh/Desktop/Đồ Án/chinh thuc/save model/vlsp_3_2_2024.pkl"

# Lưu mô hình vào tệp tin
joblib.dump(model, model_path)


['/Users/quocminh/Desktop/Đồ Án/chinh thuc/save model/vlsp_3_2_2024.pkl']

In [112]:
model.save('/Users/quocminh/Desktop/Đồ Án/chinh thuc/save model/vlsp_3_2_2024')

INFO:tensorflow:Assets written to: /Users/quocminh/Desktop/Đồ Án/chinh thuc/save model/vlsp_3_2_2024/assets


INFO:tensorflow:Assets written to: /Users/quocminh/Desktop/Đồ Án/chinh thuc/save model/vlsp_3_2_2024/assets


In [113]:
model.save("/Users/quocminh/Desktop/Đồ Án/chinh thuc/save model/vlsp_3_2_2024", save_format='tf')

INFO:tensorflow:Assets written to: /Users/quocminh/Desktop/Đồ Án/chinh thuc/save model/vlsp_3_2_2024/assets


INFO:tensorflow:Assets written to: /Users/quocminh/Desktop/Đồ Án/chinh thuc/save model/vlsp_3_2_2024/assets


In [114]:
# Lưu toàn bộ mô hình, bao gồm kiến trúc, trình tối ưu và trọng số đã học
model.save("/Users/quocminh/Desktop/Đồ Án/chinh thuc/save model/vlsp_3_2_2024.h5")

/Users/quocminh/miniconda3/lib/python3.11/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
